In [39]:
from board import *
from PIL import Image
from collections import namedtuple
from typing import Tuple
from pathlib import Path
from IPython import display
from enum import Enum
from matplotlib.pyplot import imshow
import os
from functools import reduce
from operator import iconcat

In [4]:
Coord = namedtuple('coord', 'file rank')
CropBox = Tuple[float, float, float, float]

In [5]:
class Color(Enum):
    white = 0
    black = 1

class Piece(Enum):
    bishop = 0
    knight = 1
    rook = 2
    queen = 3
    king = 4
    pawn = 5
    empty = 6 

In [29]:
class BoardImage:
    image: Image

    def __init__(self, path):
        """ only works for starting position, with white on bottom"""
        self.path = path
        self.image = Image.open(path)    
        
    _coordinates = {Piece.king: ['e1', 'e8'], 
             Piece.queen: ['d1', 'd8'], 
             Piece.knight: ['b1', 'g1', 'b8', 'g8'], 
             Piece.rook: ['a1', 'h1', 'a8', 'h8'], 
             Piece.bishop: ['c1', 'f1', 'c8', 'f8'], 
             Piece.pawn: ['a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2', 
                          'a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7'],
             Piece.empty: ['a3', 'b3', 'c3', 'd3', 'e3', 'f3', 'g3', 'h3',
                           'a4', 'b4', 'c4', 'd4', 'e4', 'f4', 'g4', 'h4',
                           'a5', 'b5', 'c5', 'd5', 'e5', 'f5', 'g5', 'h5',
                           'a6', 'b6', 'c6', 'd6', 'e6', 'f6', 'g6', 'h6']
                   }
    
    
    def _get_crop_box(self, coord: Coord) -> CropBox:
        """ 
        coord: board coordinate, a8, h3, f4, etc
        height: float of board image height
        width: float of board image width

        output: rectangle cooridates for a8, h3, f4
        """
        
        width = self.image.width
        height = self.image.height
        origin_y = (8 - int(coord.rank)) * height / 8
        origin_x = int(ord(coord.file) - ord('a')) * width / 8
        return (origin_x, origin_y, origin_x + width / 8, origin_y + height / 8)
    
    
    def get_square(self, coord):
        coord = get_coord(coord)
        cropbox = self._get_crop_box(coord)
        return self.image.crop(cropbox)
        
    def get_pieces(self, piece):
        return [self.get_square(_) for _ in self._coordinates[piece]]
    
    def get_all_pieces(self):
        raise NotImplemented        
    
    def get_black_pieces(self, color):
        raise NotImplemented
    
    def get_white_pieces(self, piece, color):
        raise NotImplemented
    
    def get_empty_squares(self):
        raise NotImplemented
    
    def __repr__(self):
        return self.path.as_posix()

In [73]:
def get_coord(name) -> Coord:
    """ name: 2 character coordinate of board square. A8, h3, f4, etc """
    file, rank = name.lower()[0], name.lower()[1]
    assert file in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    assert rank in ['1', '2', '3', '4', '5', '6', '7', '8']
    return Coord(rank=rank, file=file)

In [74]:
board = Path('data/_board/CleanShot 2021-03-28 at 21.36.07@2x.png')
board = Path('/Users/id/nassync/2051-chess-vision-blog-post/board.png')

In [75]:
bi = BoardImage(board)
pieces = reduce(iconcat, [bi.get_pieces(o) for o in Piece], [])

In [76]:
output = Path('data/blogpost/')
if not output.exists():
    output.mkdir()

In [77]:
_ = [o.save(output/f"{i}.png") for i, o in enumerate(pieces)]